In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
from config import db_username, db_password
from psycopg2 import connect, extensions, sql

### Extract CSVs into DataFrames

In [2]:
deaths_file = "Data/Deaths_FL.csv"
deaths_df = pd.read_csv(deaths_file)
# deaths_df.head()

In [3]:
all_cases_file = "Data/AllCases_FL.csv"
all_cases_df = pd.read_csv(all_cases_file)
# all_cases_df.head()

In [4]:
cases_by_county_file = "Data/AllCases_FL.csv"
cases_by_county_df = pd.read_csv(cases_by_county_file)
# cases_by_county_df.head()

In [5]:
county_population_file = "Data/CountyPopulation.csv"
county_population_df = pd.read_csv(county_population_file)
# county_population_df.head()

In [6]:
fl_schools_geo_plan_file = "Data/GeoPlan_Public_and_Private_Schools_in_Florida_-_2017.geojson"
data = json.load(open(fl_schools_geo_plan_file))
fl_schools_geo_plan_df = pd.DataFrame(data["features"])
# fl_schools_geo_plan_df.head(5)

Connect to server and create database if not exists

In [7]:
conn = connect(dbname = "", user = f"{db_username}", host = "localhost", password = f"{db_password}")

# object type: psycopg2.extensions.connection
print ("\ntype(conn):", type(conn))

# define database name
DB_NAME = "project_2_db"

# get the isolation leve for autocommit
autocommit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
print ("ISOLATION_LEVEL_AUTOCOMMIT:", extensions.ISOLATION_LEVEL_AUTOCOMMIT)

# set the isolation level for the connection's cursors
# will raise ActiveSqlTransaction exception otherwise
conn.set_isolation_level( autocommit )

# instantiate a cursor object from the connection
cursor = conn.cursor()

try:
    # use the sql module to avoid SQL injection attacks
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier( DB_NAME )))
except:
    print(f"Database {DB_NAME} Already exists")


# close the cursor to avoid memory leaks
cursor.close()

# close the connection to avoid memory leaks
conn.close()


type(conn): <class 'psycopg2.extensions.connection'>
ISOLATION_LEVEL_AUTOCOMMIT: 0


In [8]:
connection_string = f"{db_username}:{db_password}@localhost:5432/project_2_db"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
# Confirm tables
engine.table_names()

[]

### Load DataFrames into database

In [10]:
deaths_df.to_sql(name='deaths', con=engine, if_exists='replace', index=True)

In [11]:
all_cases_df.to_sql(name='all_cases', con=engine, if_exists='replace', index=True)

In [12]:
county_population_df.to_sql(name='county_population', con=engine, if_exists='replace', index=True)

In [13]:
cases_by_county_df.to_sql(name='cases_by_county', con=engine, if_exists='replace', index=True)